# Current process

main-agent  
TOI-agent -> drug name recognition  
PDF-agent -> PDF load and processing  
Info-agent -> retrieval DILI related keywords and their sections (if applicable) from the processed PDF  
DILI-agent -> determine DILI status

In [1]:
import openai
import os, re
import pandas as pd

with open('../env.txt','r') as env_file:
    for line in env_file.readlines():
        key, val = re.split('=', line.strip())
        os.environ[key]=val

from agentUtils import *
from demo_openai import MixtureOfAgents

In [2]:
pdf_dict = {
    'TAMOX': './data/hc/00064472.pdf',
    'ATORVASTATIN': './data/hc/00066863.pdf',
    'CONTRAVE': './data/hc/00072151.pdf',
    'OZEMPIC': './data/hc/00074964.pdf',
    'PANTOPRAZOLE': './data/hc/00076435.pdf',
    'SPIKEVAX': './data/hc/00077065.pdf',
    'OXYCODONE': './data/hc/00077125.pdf',
    'CONCERTA': './data/hc/00077178.pdf',

    'ATOMOXETINE': './data/hc/dili/00016602.pdf',
    'ROFACT': './data/hc/dili/00051995.pdf',
    'STIVARGA': './data/hc/dili/00055344.pdf',
    'COPAXONE': './data/hc/dili/00059892.pdf',
    'GILENYA': './data/hc/dili/00062229.pdf',
    'LEFLUNOMIDE': './data/hc/dili/00072390.pdf',
    'URSODIOL': './data/hc/dili/00072615.pdf',
    'KEYTRUDA': './data/hc/dili/00077000.pdf',

    'DOCUSATE': './data/hc/non_dili/00022698.pdf',
    'ENDOMETRIN': './data/hc/non_dili/00030499.pdf',
    'FUROSEMIDE': './data/hc/non_dili/00037990.pdf',
    'DIMENHYDRINATE': './data/hc/non_dili/00042748.pdf',
    'KETOROLAC': './data/hc/non_dili/0005900049883892.pdf',
    'DULCOLAX': './data/hc/non_dili/00057089.pdf',
    'RUZURGI': './data/hc/non_dili/00057532.pdf',
    'ESOMEPRAZOLE': './data/hc/non_dili/00059893.pdf',
    'ALENDRONATE': './data/hc/non_dili/00067108.pdf',
}

In [ ]:
# Instantiate the environment and conduct a discussion
environment = MixtureOfAgents()
client = Swarm()

bench_result=[]
for drug_name in list(pdf_dict.keys())[-2:-1]:
    response = client.run(
        agent=agent_main,  # Main agent
        messages=[{"role": "user", "content": f"What is the DILI class of {drug_name}?"}],
        context_variables={"pdf_dict":pdf_dict, "verbose":False}
    )
    input_text = response.messages[-1]["content"]
    final_ref = response.context_variables["final_reference"]
    final_kw = response.context_variables["final_keywords"]
    topic = "DILI"
    res = environment.conduct_discussion(input_text, topic)
    
    bench_result.append([drug_name, pdf_dict[drug_name], final_ref, final_kw, input_text, res])
 
final_report = pd.DataFrame(bench_result, columns=['Drug', 'File', 'Reference', 'Keywords', 'Result', 'Decision'])   


use Agent DILI
use Agent TOI
PDF Retrieval: This file ./data/hc/non_dili/00059893.pdf is used since drug name "ESOMEPRAZOLE" was found.
Discussion on topic: 'DILI' for input: 'Esomeprazole is associated with drug-induced liver injury (DILI), as indicated by warnings and precautions related to liver injury and adverse reactions like abnormal hepatic function, increased liver enzymes (SGOT, SGPT), hepatitis, jaundice, hepatic failure, and hepatic encephalopathy.'

Hepatocellular_Classifier thinks the input is ChatCompletionMessage(content='Relevant', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None) to 'DILI'.
Cholestatic_Classifier thinks the input is ChatCompletionMessage(content='Relevant', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None) to 'DILI'.
Mixed_Hepatocellular_Cholestatic_Classifier thinks the input is ChatCompletionMessage(content='Relevant', refusal=None, role='assistant', audio=None, function_call=None, tool_c

In [4]:
eval(final_report['Reference'].values[0])

{'#S1 [Warnings and Precautions]': '- DILI\n- drug-induced liver injury\n- liver injury',
 '#S2 [Adverse Reactions]': '- Drug-induced liver injury (DILI)\n- Abnormal hepatic function\n- Increased SGOT\n- Increased SGPT\n- Hepatitis\n- Jaundice\n- Hepatic failure\n- Hepatic encephalopathy'}